<a href="https://colab.research.google.com/github/merlinkalokerinos/Studies-OAMK/blob/main/ML_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip3 install scikit-learn==1.0.0

     |████████████████████████████████| 23.1 MB 1.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


In [2]:
import os # u.a. zur Entwicklugn plattformübergreifender Systempfade
import pandas as pd # Datenmanagement
import numpy as np # Hilfsfunktionen für mathematische Operationen

# Datenvisualisierung
import seaborn as sns 
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split # Datensplits
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn import metrics # Modellevaluierung

## eigene Funktionen
##def filter_df_by_proximity(df, proximity):
##    return df.loc[df["ocean_proximity"] == proximity]

##def engineer_features(df):
##    df["ratio_bedrooms"] = df["total_bedrooms"] / df["total_rooms"]
##    df["people_per_household"] = df["population"] / df["households"]
##    return df

##def get_features_and_targets(df):
##    X = df.drop(["median_house_value"], axis=1).values
##    y = np.stack(df["median_house_value"])
##    return X, y

In [14]:
df0 = pd.read_csv('/content/drive/Othercomputers/My Laptop/Data Science/final test python2.csv') # Wir lesen die Datei housing.csv ein


df1=pd.read_csv('/content/drive/Othercomputers/My Laptop/Data Science/Indikatoren München Gesamt.csv')
df01=pd.merge(df0,df1, left_on=['city district'], right_on= ['city district'])

df01.head(1)

,scraped location,purchasing price [in €],number of rooms,square metre,geo_location,city district,crime rate,Population - total,Population - german with migration background,Population - women (15 to 44),...,Migration rates - Exterior migration,Migration rates - overall migration,Migration rates by nationality - medium population (german),Migration rates by nationality - medium population (not german),Migration rates by nationality - Internal migration (german),Migration rates by nationality - Internal migration (not german),Migration rates by nationality - Exterior migration (german),Migration rates by nationality - Exterior migration (not german),Migration rates by nationality - overall migration (german),Migration rates by nationality - overall migration (not german)
0,80993 Munich,1995.0,1.0,24.0,"München, Bayern, Deutschland",10 Moosach,0.03,"54,928.00",19.4,"11,464.00",...,5.7,-4.7,"37,054.00","17,978.00",-8.2,-14.7,-13.3,44.8,-21.6,30.1


In [15]:
import os
import tarfile
import urllib.request

#df12 = pd.read_csv('/content/drive/Othercomputers/My Laptop/Data Science/final test python.csv') # Wir lesen die Datei housing.csv ein

df2= df01.drop(['scraped location','city district','geo_location']  , 1)

#del df2['scraped location']
#del df2['City district']
#del df2['Indikatoren 2021.city district']



df2 = df2.dropna() # löscht alle Zeile mit fehlenden Attributen
df2 = df2.reset_index(drop=True) # zählt unsere Daten neu durch

def get_features_and_targets(df2):
    X = df2.drop(["purchasing price [in €]"], axis=1).values
    y = np.stack(df2["purchasing price [in €]"])
    return X, y

description = df2.describe()

bins = [0] + list(description["purchasing price [in €]"][
    ["25%", "50%", "75%"]
].astype(int)) + [np.inf]

df2["house_cat"] = pd.cut(
    df2["purchasing price [in €]"],
    bins=bins,
    labels=["0 - 25%", "25 - 50%", "50 - 75%", "75 - 100%"]
)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
for train_index, test_index in split.split(df2, df2["house_cat"]):
    df2_train = df2.loc[train_index]
    df2_test = df2.loc[test_index]
    
df2_train = df2_train.drop("house_cat", axis=1)
df2_test = df2_test.drop("house_cat", axis=1)

##df_train = df_train.drop(filter_df_by_proximity(df_train, "ISLAND").index)
##df_test = df_test.drop(filter_df_by_proximity(df_test, "ISLAND").index)

##df_train = engineer_features(df_train)
##df_test = engineer_features(df_test)



df2_train_ml = pd.get_dummies(df2_train) # One-Hot Encoding
df2_test_ml = pd.get_dummies(df2_test)



X_train, y_train = get_features_and_targets(df2_train_ml)
X_test, y_test = get_features_and_targets(df2_test_ml)




clf = LinearRegression()
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


LinearRegression()

In [16]:
predictions = clf.predict(X_test)

In [17]:
print("Prediction\t|True Value")
print("-"*30)
for idx, (pred, annotation) in enumerate(zip(predictions, y_test)):
    if idx == 25:
        break
    pred = int(pred)
    annotation = int(annotation)
    print(f"{pred}\t\t|{annotation}")

Prediction	|True Value
------------------------------
1703		|1400
5071		|5100
1914		|1486
1808		|1900
1294		|1265
2370		|2500
1012		|930
2301		|2100
1196		|1250
3530		|2600
1359		|1150
949		|980
1772		|1950
1275		|1100
1564		|1915
1948		|1985
1572		|1697
2236		|2025
865		|1090
1704		|1470
4177		|4842
1280		|750
2167		|3000
1781		|1800
708		|750


In [18]:
mae = metrics.mean_absolute_error(y_test, predictions)
mae_ratio = metrics.mean_absolute_percentage_error(y_test, predictions)
r2_score = metrics.r2_score(y_test, predictions)

In [19]:

print(f"Wir können durchschnittlich mit einem Fehler von {np.round(mae)} rechnen;")
print(f"Das entspricht im Schnitt einer Fehlerquote von {np.round(mae_ratio * 100)}%;")
print(f"Der R^2 Werte liegt bei {np.round(r2_score * 100)}%")

Wir können durchschnittlich mit einem Fehler von 309.0 rechnen;
Das entspricht im Schnitt einer Fehlerquote von 21.0%;
Der R^2 Werte liegt bei 63.0%


In [20]:
df2_test.iloc[0]


purchasing price [in €]                                                  1400.0
number of rooms                                                             2.0
square metre                                                               74.0
crime rate                                                                 0.04
Population  - total                                                  75,658.00 
                                                                       ...     
Migration rates by nationality - Internal migration (not german)           -1.4
Migration rates by nationality - Exterior migration (german)              -16.7
Migration rates by nationality - Exterior migration (not german)           25.8
Migration rates by nationality - overall migration (german)               -31.5
Migration rates by nationality - overall migration (not german)            24.4
Name: 1130, Length: 114, dtype: object

In [ ]:


single_pred = clf.predict([X_test[0]])[0]
print(f"Die Prognose liegt bei {single_pred}.")
print(f"Der Realwert war {y_test[0]}.")


In [ ]:

df2_test['predictions']=predictions
#print(df2_test)

df2_test.append([df2_train])
df2_test.head(6000)
#df12.head(6000)

In [ ]:
dffinal=pd.merge(df12, df2_test, left_index=True, right_index=True, how='left')

In [ ]:
dffinal.to_csv('/content/drive/Othercomputers/My Laptop/Data Science/ML Python/python export predictions ohne indikatoren.csv')